In [1]:
#导入第三方包
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import scipy.stats.mstats as mst

import matplotlib as mpl


plt.style.use('ggplot')

#设定目录及读取数据文件
dir = './data/prophet/'
train = pd.read_table(dir + 'brand4.csv',engine='python',sep=',')
df2=pd.read_table('./data/train2.csv',engine='python',sep=',')
df2.head()

,date,day_of_week,cnt
0,1,3,68
1,2,4,36
2,3,5,5565
3,4,6,4966
4,5,7,3346


In [2]:
df=train
df2=df2.loc[:,['date','day_of_week']]
df3=pd.merge(df2,train,on=['date','day_of_week'],how='outer')
df3=df3.sort_values(by='date')
df3.head(40)
df3['brand']=df3['brand'].fillna(5)
df3['cnt']=df3['cnt'].fillna(0)
df3.head(40)

,date,day_of_week,brand,cnt
0,1,3,5.0,0.0
1,2,4,5.0,0.0
2,3,5,4.0,773.0
3,4,6,4.0,1057.0
4,5,7,4.0,759.0
5,6,1,4.0,725.0
6,7,2,4.0,1242.0
7,8,3,4.0,648.0
8,9,4,4.0,554.0
9,10,5,4.0,459.0


In [3]:
#!usr/bin/env python
#-*- coding: utf-8 -*-
#回归base model初选

import sys
import os
import time
from sklearn import metrics
import numpy as np
from sklearn import model_selection
import datetime
import sklearn
import datetime


#reload(sys)
#sys.setdefaultencoding('utf8')





# Logistic Regression Classifier
def logistic_regression_Regressor(train_x, train_y):
    from sklearn.linear_model import Ridge
    model = Ridge(alpha=29)
    model.fit(train_x, train_y)
    return model


# Random Forest Classifier
def random_forest_Regressor(train_x, train_y):
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(n_jobs=-1,n_estimators=11)
    model.fit(train_x, train_y)
    return model


# Decision Tree Classifier
def decision_tree_Regressor(train_x, train_y):
    from sklearn import tree
    model = tree.DecisionTreeClassifier(criterion='entropy')
    model.fit(train_x, train_y)
    return model

# AdaBoostRegressor
def AdaBoostRegressor(train_x, train_y):
    from sklearn.ensemble import AdaBoostClassifier
    model = AdaBoostClassifier(n_estimators=144)
    model.fit(train_x, train_y)
    return model
# Xgboost Classfier
def XGBClassifier(train_x, train_y):
    from xgboost.sklearn import XGBClassifier
    model = XGBClassifier(nthread=7,learning_rate=0.06)
    model.fit(train_x, train_y)
    return model

#分割数据集，生成测试和训练集
def read_data(df):
    data=df[['day_of_week','date']]
    target=df['cnt']
    train_x, test_x, train_y, test_y= model_selection.train_test_split(data,target,test_size=0.28,random_state=1)
    return train_x, test_x, train_y, test_y,

if __name__ == '__main__':

    test_Regressors = ['LR','RF', 'DT','XGB']
    Regressors = {'LR':logistic_regression_Regressor,
   'RF':random_forest_Regressor,
   'DT':decision_tree_Regressor,
   'ADA':AdaBoostRegressor,'XGB':XGBClassifier
    }




    print('******************** Strating *********************')


    starttime = datetime.datetime.now()
    train_x, test_x, train_y, test_y = read_data(df3)
    for Regressor in test_Regressors:
        print('******************* %s ********************' % Regressor)
        start_time = time.time()
        model = Regressors[Regressor](train_x, train_y)	
        print('training took %fs!' % (time.time() - start_time))
        predict = model.predict(test_x)
        #p=predict.round(0)
        #模型拟合测试集
        from sklearn import metrics
        name=''
        name=Regressor
        # 用scikit-learn计算MSE
        mse=metrics.mean_squared_error(test_y, predict)
        print(mse)
        from sklearn.externals import joblib
        joblib.dump(model, dir+name+'_brand4.pkl')

    endtime = datetime.datetime.now()
    print('training took %fs!' % (endtime - starttime).seconds)
    df.head()

******************** Strating *********************
******************* LR ********************
training took 1.841504s!
35212.8413425
******************* RF ********************
training took 0.425501s!
48627.5847751
******************* DT ********************
training took 0.020000s!
44247.2733564
******************* XGB ********************
training took 2.569505s!
43743.7750865
training took 5.000000s!


In [4]:
from sklearn import cross_validation
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from xgboost.sklearn import XGBClassifier
clf= AdaBoostClassifier()
parameter_space = {"n_estimators":range(135,145)}   
grid = GridSearchCV(clf, param_grid=parameter_space,scoring='neg_mean_squared_error',cv=5,n_jobs=6)  
grid.fit(test_x, test_y)
print(grid.best_params_)
print(grid.score)

d:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
d:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


{'n_estimators': 141}
<bound method BaseSearchCV.score of GridSearchCV(cv=5, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params=None, iid=True, n_jobs=6,
       param_grid={'n_estimators': range(135, 145)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)>


In [5]:
from sklearn.linear_model import RidgeCV
# 在初始化RidgeCV类时, 提供一组备选的α值,RidgeCV类会帮我们选择一个合适的α值.
ridgecv = RidgeCV(alphas=range(12,30), cv=5)
# 拟合训练集
ridgecv.fit(train_x,train_y)
# 打印最优的α值
print(ridgecv.alpha_)
test_Y_pred =ridgecv.predict(test_x)
auc=metrics.mean_squared_error(test_y,test_Y_pred)
print(auc)

21
35208.5334614
